# UTMB Weather Weighted Average Pace Prediction

Weather is a key factor into the performance of runners. It
seems that in extreme heat, the performance of athletes decreases. This would also be relative
to ultra marathons. The mean yearly average pace was compared to the mean yearly average
temperature and, using linear regression, found that as the degrees Celsius temperature rises by 1
degree, there is a 0.1 rise in minutes per kilometre pace for the UTMB and Western States. Each
race was imagined to be ran at 22 degrees Celsius. Therefore, for 2016, where the temperature
was 30 degrees Celsius, (0.1 * 8) was subtracted from every average pace of every runner for
that year. This process was repeated for each year, multiplying 0.1 by the difference between the
temperatures. The prediction model was tested using these new weather weighted times.


In [938]:
import os.path
import datetime
import pandas as pd
import numpy as np
import hashlib
import matplotlib
from scipy.spatial.distance import cosine
from scipy.spatial.distance import cdist
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform, pdist

%matplotlib inline

# Display all columns in Pandas
pd.set_option('display.max_columns', None) 

I aim to produce a challenging but not impossible average pace prediction for each runner. I will first get the runner's non personal best average pace and find the most similar non personal best average pace to that runner. I will then find the personal best average pace of these runners. By average these pb times, or getting the best personal best average pace of these runners, I will be able to produce a predicted average pace. 

In [939]:
# Constants
pace_threshold = 1
#euclidean_distance_threshold = 200

In [940]:
# Read in Split Timings
df = pd.read_csv('../../data/1km_splits_full.csv', encoding = "ISO-8859-1")

In [941]:
# Drop unnecessary columns
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'bib','team', 'rank', 'nationality'], axis = 1)

In [942]:
imp_cats = df['category']
imp_year = df['Year']
imp_name = df['name_mask']
imp_gen = df['gender']
imp_comb = [imp_cats, imp_gen, imp_name, imp_year]
imp_comb = pd.concat(imp_comb, axis=1)

In [943]:
df.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
0,2017,7efc0cb7e70d7cd9da46a58bbf01af66,Male,Senior Men (23-39),5.281863,NaN,NaN,4.375000,NaN,5.443366,6.483539,9.900000,4.980392,8.205502,4.044218,9.433333,5.237037,5.646341,NaN,NaN,10.649660,6.691441,6.487179,11.881481,5.323333,5.707143,NaN,NaN,8.030973,NaN,NaN,6.102041,NaN,8.604167,NaN,7.747126,NaN,NaN,7.788288,NaN,NaN,9.016667,5.236486,1141.900000,NaN,7.012357
1,2017,1e4ba9a22b3fecb9136ecfcf7f470868,Male,Senior Men (23-39),5.147059,NaN,NaN,4.519737,NaN,5.501618,6.510288,10.100000,4.970588,8.098706,4.047619,9.429167,5.255556,5.743902,NaN,NaN,11.159864,6.887387,6.810897,12.225926,5.376667,5.800000,NaN,NaN,8.781711,NaN,NaN,5.768707,NaN,8.729167,NaN,7.235632,NaN,NaN,7.869369,NaN,NaN,9.300000,5.072072,1156.983333,15.083333,7.097568
2,2017,0d928f7482046900e19dba217d7befa5,Male,Senior Men (23-39),5.544118,NaN,NaN,4.372807,NaN,5.500000,6.981481,10.769697,5.689542,8.812298,4.452381,10.020833,5.629630,6.292683,NaN,NaN,11.411565,6.979730,6.519231,NaN,11.085000,5.870238,NaN,NaN,8.631268,NaN,NaN,5.323129,NaN,9.222222,NaN,7.497126,NaN,NaN,7.918919,NaN,NaN,9.804762,4.772523,1193.000000,51.100000,7.352225
3,2017,93f20063908ba5d48724dcf31740a52a,Male,Senior Men (23-39),5.283088,NaN,NaN,4.379386,NaN,5.441748,6.489712,11.700000,5.741830,9.105178,4.812925,10.220833,5.677778,6.516260,NaN,NaN,12.340136,7.243243,7.243590,13.255556,5.170000,5.839286,NaN,NaN,8.510324,NaN,NaN,6.119048,NaN,9.104167,NaN,7.370690,NaN,NaN,7.585586,NaN,NaN,9.238095,4.808559,1203.650000,61.750000,7.466542
4,2017,7479fb90beffad048c5f772ba1915b4e,Male,Senior Men (23-39),5.145833,NaN,NaN,4.245614,NaN,5.582524,6.615226,9.887879,4.973856,8.249191,3.901361,8.366667,5.162963,6.024390,NaN,NaN,10.870748,7.063063,6.448718,11.896296,5.868333,7.513095,NaN,NaN,12.036873,NaN,NaN,4.996599,NaN,8.180556,NaN,6.422414,NaN,NaN,8.414414,NaN,NaN,10.057143,5.141892,1211.633333,69.733333,7.211069


##  Weighting pace according to weather

1 degree rise in temperature causes a 0.1 rise in average pace. Weighted times as if all races were ran in 22 degrees.  So for example in 2017, the temperature was 17 degrees (5 degrees less than 22) so 5 * 0.1 is added to the Average Pace of each runner

In [944]:
df17 = df[df.Year == 2017]

In [945]:
df17 = df17.loc[:, 'Chamonix to Delevret':'AVG_Pace'] + 0.5

In [946]:
df16 = df[df.Year == 2016]

In [947]:
df16 = df16.loc[:, 'Chamonix to Delevret':'AVG_Pace'] - 0.8

In [948]:
df15 = df[df.Year == 2015]

In [949]:
df15 = df15.loc[:, 'Chamonix to Delevret':'AVG_Pace'] - 0.3

In [950]:
df14 = df[df.Year == 2014]

In [951]:
df14 = df14.loc[:, 'Chamonix to Delevret':'AVG_Pace']

In [952]:
df13 = df[df.Year == 2013]

In [953]:
df13 = df13.loc[:, 'Chamonix to Delevret':'AVG_Pace'] + 0.1

In [954]:
df11 = df[df.Year == 2011]

In [955]:
df11 = df11.loc[:, 'Chamonix to Delevret':'AVG_Pace']  - 0.1

In [956]:
year_frames = [df17, df16, df15, df14, df13, df11]
df = pd.concat(year_frames)

In [957]:
df = df.join(imp_comb)

In [958]:
df = df[['Year', 
'name_mask'           ,
'gender',
'category'        , 
         'time',
'Chamonix to Delevret'            ,
#'Chamonix to Col de Voza'          ,
'Chamonix to La Charme'          ,
'Chamonix to St-Gervais'          ,
'Delevret to St-Gervais'            ,
#'Col de Voza to Contamines'          ,
'La Charme to St-Gervais'          ,
'St-Gervais to Contamines'          ,
'Contamines to La Balme'            ,
#'Contamines to Bonhomme'          ,
'La Balme to Bonhomme'          ,
#'La Balme to Chapieux'          ,
'Bonhomme to Chapieux'          ,
'Chapieux to Col Seigne'            ,
#'Col Seigne to Refuge Elisabeth'          ,
'Col Seigne to Lac Combal'          ,
#'Col Seigne to Mt-Favre'          ,
#'Refuge Elisabeth to Mt-Favre'         ,
#'Refuge Elisabeth to Checruit'        ,
'Lac Combal to Mt-Favre'         ,
'Mt-Favre to Checruit'         ,
#'Mt-Favre to Courmayeur'        ,
'Checruit to Courmayeur'        ,
'Courmayeur to Courmayeur2'          ,
'Courmayeur2 to Bertone'          ,
'Courmayeur to Bertone'        ,
#'Bertone to Refuge Elena'         ,
'Bertone to Bonatti'         ,
#'Refuge Elena to Col Ferret'          ,
'Bonatti to Arnouvaz'        ,
'Arnouvaz to Col Ferret'         ,
#'Col Ferret to La Peulaz'         ,
'Col Ferret to La Fouly'        ,
#'La Peulaz to La Fouly'          ,
#'La Fouly to Praz de Fort'     ,
'La Fouly to Champex La'        ,
#'Praz de Fort to Champex La'        ,
#'Praz de Fort to Champex den Bas',
'Champex La to Bovine'        ,
'Champex La to Mortigny'        ,
'Champex La to Giete'        ,
#'Champex den Bas to Bovine'        ,
'Bovine to Trient'         ,
'Mortigny to Trient'         ,
'Giete to Trient'          ,
'Trient to La Catogne'        ,
'Trient to Les Tseppe'        ,
#'Trient to Vallorcine'        ,
'La Catogne to Vallorcine'         ,
'Les Tseppe to Vallorcine'         ,
#'Vallorcine to La Gardes'         ,
'Vallorcine to Tete aux Vents'         ,
'Vallorcine to Argentiere'         ,
'Vallorcine to Col Montet'         ,
#'La Gardes to Chamonix'        ,
'Tete aux Vents to Flegere'          ,
'Argentiere to Chamonix'     ,
'Col Montet to Flegere'        ,
'Flegere to Chamonix', 
        'AVG_Pace']]

In [959]:
df.head()

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
0,2017,7efc0cb7e70d7cd9da46a58bbf01af66,Male,Senior Men (23-39),1142.400000,5.781863,NaN,NaN,4.875000,NaN,5.943366,6.983539,10.400000,5.480392,8.705502,4.544218,9.933333,5.737037,6.146341,NaN,NaN,11.149660,7.191441,6.987179,12.381481,5.823333,6.207143,NaN,NaN,8.530973,NaN,NaN,6.602041,NaN,9.104167,NaN,8.247126,NaN,NaN,8.288288,NaN,NaN,9.516667,5.736486,7.512357
1,2017,1e4ba9a22b3fecb9136ecfcf7f470868,Male,Senior Men (23-39),1157.483333,5.647059,NaN,NaN,5.019737,NaN,6.001618,7.010288,10.600000,5.470588,8.598706,4.547619,9.929167,5.755556,6.243902,NaN,NaN,11.659864,7.387387,7.310897,12.725926,5.876667,6.300000,NaN,NaN,9.281711,NaN,NaN,6.268707,NaN,9.229167,NaN,7.735632,NaN,NaN,8.369369,NaN,NaN,9.800000,5.572072,7.597568
2,2017,0d928f7482046900e19dba217d7befa5,Male,Senior Men (23-39),1193.500000,6.044118,NaN,NaN,4.872807,NaN,6.000000,7.481481,11.269697,6.189542,9.312298,4.952381,10.520833,6.129630,6.792683,NaN,NaN,11.911565,7.479730,7.019231,NaN,11.585000,6.370238,NaN,NaN,9.131268,NaN,NaN,5.823129,NaN,9.722222,NaN,7.997126,NaN,NaN,8.418919,NaN,NaN,10.304762,5.272523,7.852225
3,2017,93f20063908ba5d48724dcf31740a52a,Male,Senior Men (23-39),1204.150000,5.783088,NaN,NaN,4.879386,NaN,5.941748,6.989712,12.200000,6.241830,9.605178,5.312925,10.720833,6.177778,7.016260,NaN,NaN,12.840136,7.743243,7.743590,13.755556,5.670000,6.339286,NaN,NaN,9.010324,NaN,NaN,6.619048,NaN,9.604167,NaN,7.870690,NaN,NaN,8.085586,NaN,NaN,9.738095,5.308559,7.966542
4,2017,7479fb90beffad048c5f772ba1915b4e,Male,Senior Men (23-39),1212.133333,5.645833,NaN,NaN,4.745614,NaN,6.082524,7.115226,10.387879,5.473856,8.749191,4.401361,8.866667,5.662963,6.524390,NaN,NaN,11.370748,7.563063,6.948718,12.396296,6.368333,8.013095,NaN,NaN,12.536873,NaN,NaN,5.496599,NaN,8.680556,NaN,6.922414,NaN,NaN,8.914414,NaN,NaN,10.557143,5.641892,7.711069


In [960]:
df.groupby('category').count()

,Year,name_mask,gender,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Masters Men 1 (40-49),6007,6007,6007,3846,5994,0,0,5990,0,5901,5686,5654,5647,5462,5448,5264,5282,5246,1697,816,4045,4799,4802,4472,4454,4131,697,540,2726,691,534,2722,3164,714,3167,714,2623,524,712,2620,523,711,3323,6004
Masters Men 2 (50-59),2597,2597,2597,1381,2592,0,0,2588,0,2531,2410,2388,2386,2270,2256,2172,2170,2161,675,330,1585,1882,1884,1704,1694,1523,295,183,974,292,183,970,1178,239,1176,239,980,173,235,972,172,236,1209,2595
Masters Men 3 (60-69),372,372,372,123,371,0,0,370,0,357,328,326,326,305,302,277,277,272,70,35,189,217,218,174,169,143,36,23,76,35,23,77,117,13,116,13,89,22,13,88,21,13,102,372
Masters Men 4 (70-79),23,23,23,2,23,0,0,23,0,22,21,20,20,18,18,16,16,15,2,1,9,10,10,7,7,3,0,0,2,0,0,2,2,0,2,0,2,0,0,2,0,0,2,23
Masters Women 1 (40-49),564,564,564,318,560,0,0,560,0,547,521,515,518,497,493,476,476,472,166,72,351,422,417,373,371,342,58,29,240,58,29,239,261,61,263,61,230,27,61,231,27,61,291,562
Masters Women 2 (50-59),222,222,222,106,220,0,0,220,0,214,201,200,198,186,183,176,177,172,51,23,128,149,149,133,132,118,29,12,69,29,12,69,88,20,87,20,74,12,20,73,12,20,94,220
Masters Women 3 (60-69),21,21,21,6,21,0,0,21,0,19,16,17,16,14,14,13,13,12,8,5,5,10,10,7,7,6,1,1,4,1,1,4,6,0,6,0,5,1,0,5,1,0,5,21
Senior Men (23-39),4569,4569,4569,3083,4555,0,0,4555,0,4493,4356,4329,4328,4208,4211,4091,4102,4092,1325,639,3210,3815,3812,3622,3611,3352,574,367,2247,564,355,2248,2523,586,2523,586,2159,344,580,2159,344,580,2739,4563
Senior Women (23-39),521,521,521,306,519,0,0,518,0,506,495,488,487,470,466,450,451,452,156,70,346,406,404,372,369,341,57,33,237,57,33,234,250,66,248,66,211,32,66,211,32,66,274,520


In [962]:
df[(df["AVG_Pace"] <12) & (df.category == 'Masters Women 2 (50-59)')]


,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
2471,2017,6edbc123d2c45fa61b34a1f47eae4a2c,Female,Masters Women 2 (50-59),NaN,10.767157,NaN,NaN,10.118421,NaN,11.941748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.942442
2485,2017,dd9e346cc77bdb575a062caf1e045922,Female,Masters Women 2 (50-59),NaN,11.779412,NaN,NaN,11.245614,NaN,12.741100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.922042
2595,2016,36da23ac15e20c3b3e35b879229da995,Female,Masters Women 2 (50-59),1753.133333,7.149755,NaN,NaN,5.044298,NaN,7.963754,9.113580,12.948485,6.572549,9.780906,14.108163,12.458333,6.766667,8.139024,17.255556,NaN,13.958503,8.116667,7.850641,15.111111,6.043333,7.825,NaN,NaN,10.720649,NaN,NaN,7.764626,15.806918,NaN,7.394444,NaN,16.340693,NaN,NaN,13.543434,NaN,NaN,7.697748,10.218994
4615,2016,91e2e960faf46539022b23717f8575de,Female,Masters Women 2 (50-59),NaN,6.282108,NaN,NaN,5.467544,NaN,7.394175,10.060082,13.709091,8.445098,10.844013,18.631973,23.050000,11.959259,13.947967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.799210
5066,2016,518c744fb20eb7d4c43afd66c1410b57,Female,Masters Women 2 (50-59),NaN,10.008824,NaN,NaN,8.178070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.093447
5079,2016,8cb28bf1befd988376fe7152453f08e9,Female,Masters Women 2 (50-59),NaN,10.364216,NaN,NaN,10.450000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.407108
5081,2016,85d4d22c35b487dcb6fa7c4fcfca9ae3,Female,Masters Women 2 (50-59),NaN,12.636275,NaN,NaN,8.283333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.459804
7405,2015,cbf15f23ef5ead61ed9491f0e92aa9b8,Female,Masters Women 2 (50-59),NaN,8.791912,NaN,NaN,7.408333,NaN,9.585113,13.932510,17.996970,9.611765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.221101
7546,2015,ea16550cfbbfd6cd4327356dc80126e6,Female,Masters Women 2 (50-59),NaN,8.700000,NaN,NaN,8.763596,NaN,11.615858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.693151
7559,2015,7c2c1575d30df2b22d3662dc49bc41bb,Female,Masters Women 2 (50-59),NaN,9.867892,NaN,NaN,8.537719,NaN,11.411974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.939195


In [963]:
#df = df.drop([7767])
#df = df.drop([10182])

In [964]:
df.loc[1613]

Year                                                        2017
name_mask                       4db319a671ab6e2b099df28df89f85c4
gender                                                      Male
category                                   Masters Men 1 (40-49)
time                                                     2722.33
Chamonix to Delevret                                      9.1826
Chamonix to La Charme                                        NaN
Chamonix to St-Gervais                                       NaN
Delevret to St-Gervais                                   8.61842
La Charme to St-Gervais                                      NaN
St-Gervais to Contamines                                 11.2799
Contamines to La Balme                                   15.2181
La Balme to Bonhomme                                     23.2212
Bonhomme to Chapieux                                     11.3431
Chapieux to Col Seigne                                   18.4725
Col Seigne to Lac Combal 

In [965]:
# Filter out NaN times
df = df.dropna(subset=['time'])
# Sort by name (first) and year (second)
# Sorting by year, since it might make sense to later only generate prediction for latest non-pb time
df.sort_values(['name_mask', 'Year'], ascending=[True, False], inplace=True)
# Show sample of data
df.head()

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
11228,2013,0001fff82652ce3818504f26771297a3,Male,Senior Men (23-39),2587.033333,8.025245,NaN,NaN,6.711842,NaN,10.415534,10.577366,16.090909,10.711111,15.663107,9.858503,22.125000,11.270370,13.502439,NaN,NaN,33.287075,12.685586,12.090385,23.570370,10.975000,13.605952,21.399663,NaN,NaN,16.097354,NaN,NaN,27.405031,NaN,18.233333,NaN,27.887879,NaN,NaN,18.145455,NaN,NaN,18.214865,16.189557
3623,2016,0002b9e98b5761bb4aa0960616f45ca6,Male,Masters Men 1 (40-49),2645.366667,7.659559,NaN,NaN,6.853509,NaN,11.049515,16.341975,17.669697,11.817647,15.920065,23.244218,16.537500,10.222222,12.854472,65.644444,NaN,17.325850,13.506306,11.866667,22.718519,10.510000,16.888095,NaN,NaN,18.747198,NaN,NaN,13.791837,27.366667,NaN,10.730556,NaN,18.920779,NaN,NaN,13.639394,NaN,NaN,10.535586,16.894491
5634,2015,000bf78327bdfa8c6298fc9a76921305,Male,Masters Men 1 (40-49),2323.450000,7.671814,NaN,NaN,6.357895,NaN,9.149838,12.652675,16.909091,10.023529,13.873139,21.114966,18.254167,9.970370,11.496748,35.477778,23.316667,NaN,12.672973,12.565385,23.062963,10.835000,12.334524,NaN,NaN,17.390265,NaN,NaN,12.063946,20.027044,NaN,12.208333,NaN,17.438095,NaN,NaN,15.720202,NaN,NaN,10.486036,14.922938
7880,2014,000d01e06e9abfe51cdb08ff2567f125,Male,Senior Men (23-39),2021.466667,7.726716,NaN,NaN,6.125000,NaN,8.590615,10.203704,14.993939,7.905229,11.296117,7.071429,15.108333,8.466667,10.317073,NaN,NaN,21.159864,10.364865,10.740385,18.259259,9.588333,11.288095,NaN,NaN,15.786136,NaN,NaN,11.030612,18.198113,NaN,13.405556,NaN,18.824675,NaN,NaN,18.515152,NaN,NaN,13.027027,12.416371
8786,2014,001e405237215c01ebb54168bc0d93ec,Male,Masters Men 2 (50-59),2627.083333,8.161765,NaN,NaN,8.004386,NaN,10.145631,12.958848,18.939394,10.480392,14.407767,10.850340,20.237500,10.292593,14.182927,NaN,NaN,27.693878,12.916667,15.291667,23.518519,12.421667,13.336905,NaN,NaN,23.163717,NaN,NaN,19.513605,25.518868,NaN,18.911111,NaN,25.900433,NaN,NaN,21.712121,NaN,NaN,13.033784,16.316437


# Filter out runners who only ran one race

Runner's have ran at least 2 races to be included in the case base, ie. a non personal best time and a personal best time

In [966]:
# Count number of races per runner
v = df.name_mask.value_counts()
# Get runners who ran more than one race
multi_race_runners = v.index[v.gt(1)]
# Filter by multi_race_runners
df = df[df.name_mask.isin(multi_race_runners)]

In [967]:
df.head()

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
8150,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2312.483333,9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,14.183180
10507,2013,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2183.416667,9.475000,NaN,NaN,6.922368,NaN,10.046602,11.295473,16.930303,8.332026,13.920388,7.668027,17.250000,8.637037,10.173171,NaN,NaN,22.579592,10.728378,11.035897,23.777778,7.825000,12.182143,13.344108,NaN,NaN,12.647619,NaN,NaN,19.703774,NaN,15.233333,NaN,22.316450,NaN,NaN,18.892929,NaN,NaN,15.230631,13.589501
13323,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2581.233333,10.594853,NaN,NaN,10.560088,NaN,11.344984,14.690123,18.672727,10.256209,14.532686,9.376190,17.733333,10.933333,13.152033,NaN,NaN,25.886395,11.713063,16.156410,25.733333,15.486667,17.373810,NaN,20.975426,NaN,NaN,16.459829,NaN,21.488050,NaN,11.588889,NaN,NaN,10.667196,NaN,NaN,11.662452,NaN,NaN,15.088612
3763,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),2703.116667,8.112990,NaN,NaN,6.386404,NaN,9.622330,13.158848,18.600000,10.487582,15.763107,25.271429,15.537500,9.570370,10.208130,47.2,NaN,21.417687,13.706757,16.427564,20.859259,10.913333,13.300000,NaN,NaN,21.288496,NaN,NaN,14.502721,24.690566,NaN,12.572222,NaN,25.139394,NaN,NaN,20.159596,NaN,NaN,15.157207,16.802140
8431,2014,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Senior Men (23-39),2456.833333,8.466912,NaN,NaN,7.383772,NaN,8.750809,12.213992,16.269697,10.627451,14.220065,9.401361,17.579167,10.533333,10.540650,NaN,NaN,29.227891,11.637387,15.320513,23.751852,11.983333,14.132143,NaN,NaN,20.764012,NaN,NaN,15.326531,26.301887,NaN,16.358333,NaN,23.008658,NaN,NaN,18.101010,NaN,NaN,11.027027,15.121991


# Merge

In [968]:
# Get Personal Best
pb = df[df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]
# Drop Year and category to avoid duplication on merge
pb = pb.drop(['Year', 'category'], axis = 1)
# Get Non-Personal Best Times
npb = df[~df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]

In [969]:
pb.head()

,name_mask,gender,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
10507,003cc0338235bb1862eb7b9bdd31ec9c,Male,2183.416667,9.475000,NaN,NaN,6.922368,NaN,10.046602,11.295473,16.930303,8.332026,13.920388,7.668027,17.250000,8.637037,10.173171,NaN,NaN,22.579592,10.728378,11.035897,23.777778,7.825000,12.182143,13.344108,NaN,NaN,12.647619,NaN,NaN,19.703774,NaN,15.233333,NaN,22.316450,NaN,NaN,18.892929,NaN,NaN,15.230631,13.589501
8431,00a7ade381ebb6d25e1aacb0bc7129e9,Male,2456.833333,8.466912,NaN,NaN,7.383772,NaN,8.750809,12.213992,16.269697,10.627451,14.220065,9.401361,17.579167,10.533333,10.540650,NaN,NaN,29.227891,11.637387,15.320513,23.751852,11.983333,14.132143,NaN,NaN,20.764012,NaN,NaN,15.326531,26.301887,NaN,16.358333,NaN,23.008658,NaN,NaN,18.101010,NaN,NaN,11.027027,15.121991
6180,011b597a8c4121461b659f50dbd19d5a,Male,2594.800000,8.111765,NaN,NaN,7.417105,NaN,10.208091,12.413992,19.266667,11.023529,14.892557,23.220408,18.791667,11.070370,11.923577,29.033333,27.475000,NaN,15.308108,16.235256,24.625926,10.751667,14.061905,NaN,NaN,20.912389,NaN,NaN,12.740816,24.894969,NaN,12.330556,NaN,21.654545,NaN,NaN,17.629293,NaN,NaN,12.161712,16.326208
6418,01368b79f758adce8d638a35b80c8a99,Male,2691.966667,7.839706,NaN,NaN,6.164912,NaN,9.394175,12.763786,17.118182,9.997386,16.763107,24.523129,22.962500,11.662963,15.659350,60.033333,24.970833,NaN,14.623423,15.655128,23.637037,11.198333,15.998810,NaN,NaN,21.316519,NaN,NaN,13.346259,28.514465,NaN,11.900000,NaN,21.643723,NaN,NaN,17.775758,NaN,NaN,13.675225,17.965522
10262,013e5d702101486cbcb28f8265f2d30c,Male,1918.933333,8.517892,NaN,NaN,6.409211,NaN,9.774757,9.157613,15.442424,9.145752,12.106472,7.589796,16.154167,8.818519,10.913008,NaN,NaN,23.321088,10.410811,10.664103,16.833333,9.040000,10.170238,13.307071,NaN,NaN,11.917460,NaN,NaN,18.562264,NaN,8.683333,NaN,17.210390,NaN,NaN,11.302020,NaN,NaN,9.660811,11.879689


In [970]:
npb.head()

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
8150,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2312.483333,9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,14.183180
13323,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2581.233333,10.594853,NaN,NaN,10.560088,NaN,11.344984,14.690123,18.672727,10.256209,14.532686,9.376190,17.733333,10.933333,13.152033,NaN,NaN,25.886395,11.713063,16.156410,25.733333,15.486667,17.373810,NaN,20.975426,NaN,NaN,16.459829,NaN,21.488050,NaN,11.588889,NaN,NaN,10.667196,NaN,NaN,11.662452,NaN,NaN,15.088612
3763,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),2703.116667,8.112990,NaN,NaN,6.386404,NaN,9.622330,13.158848,18.600000,10.487582,15.763107,25.271429,15.537500,9.570370,10.208130,47.2,NaN,21.417687,13.706757,16.427564,20.859259,10.913333,13.300000,NaN,NaN,21.288496,NaN,NaN,14.502721,24.690566,NaN,12.572222,NaN,25.139394,NaN,NaN,20.159596,NaN,NaN,15.157207,16.802140
11265,2013,011b597a8c4121461b659f50dbd19d5a,Male,Masters Men 1 (40-49),2602.466667,9.005637,NaN,NaN,8.174561,NaN,11.711650,11.610288,21.154545,11.524837,16.530421,11.491156,23.391667,11.670370,12.002439,NaN,NaN,26.926531,12.949099,13.702564,23.129630,12.508333,12.977381,18.426599,NaN,NaN,19.700529,NaN,NaN,29.606289,NaN,16.152778,NaN,25.409524,NaN,NaN,15.660606,NaN,NaN,14.259910,16.236556
1520,2017,01368b79f758adce8d638a35b80c8a99,Male,Senior Men (23-39),2699.366667,9.366422,NaN,NaN,8.320175,NaN,10.805825,12.833333,19.327273,13.058824,16.778317,11.306122,23.329167,12.470370,15.512195,NaN,NaN,35.234694,15.292793,15.272436,27.085185,11.908333,18.986905,NaN,NaN,22.383481,NaN,NaN,18.176871,NaN,21.732639,NaN,19.284483,NaN,NaN,21.981982,NaN,NaN,19.604762,15.479730,17.313847


In [971]:
npb[npb.AVG_Pace.isnull()]

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace


In [972]:
# Merge to get non-pb runs with personal best time (with time, timediff and AVG_Pace)
merge = npb.merge(pb,how='left',on='name_mask',suffixes=('', '_pb'))

In [973]:
#merge = merge.groupby('name_mask').head(1)

In [974]:
first_df = len(merge)

In [975]:
first_df

1006

In [885]:
merge.head()

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2312.483333,9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,14.183180,Male,2183.416667,9.475000,NaN,NaN,6.922368,NaN,10.046602,11.295473,16.930303,8.332026,13.920388,7.668027,17.250000,8.637037,10.173171,NaN,NaN,22.579592,10.728378,11.035897,23.777778,7.825000,12.182143,13.344108,NaN,NaN,12.647619,NaN,NaN,19.703774,NaN,15.233333,NaN,22.316450,NaN,NaN,18.892929,NaN,NaN,15.230631,13.589501
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2581.233333,10.594853,NaN,NaN,10.560088,NaN,11.344984,14.690123,18.672727,10.256209,14.532686,9.376190,17.733333,10.933333,13.152033,NaN,NaN,25.886395,11.713063,16.156410,25.733333,15.486667,17.373810,NaN,20.975426,NaN,NaN,16.459829,NaN,21.488050,NaN,11.588889,NaN,NaN,10.667196,NaN,NaN,11.662452,NaN,NaN,15.088612,Male,2183.416667,9.475000,NaN,NaN,6.922368,NaN,10.046602,11.295473,16.930303,8.332026,13.920388,7.668027,17.250000,8.637037,10.173171,NaN,NaN,22.579592,10.728378,11.035897,23.777778,7.825000,12.182143,13.344108,NaN,NaN,12.647619,NaN,NaN,19.703774,NaN,15.233333,NaN,22.316450,NaN,NaN,18.892929,NaN,NaN,15.230631,13.589501
2,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),2703.116667,8.112990,NaN,NaN,6.386404,NaN,9.622330,13.158848,18.600000,10.487582,15.763107,25.271429,15.537500,9.570370,10.208130,47.2,NaN,21.417687,13.706757,16.427564,20.859259,10.913333,13.300000,NaN,NaN,21.288496,NaN,NaN,14.502721,24.690566,NaN,12.572222,NaN,25.139394,NaN,NaN,20.159596,NaN,NaN,15.157207,16.802140,Male,2456.833333,8.466912,NaN,NaN,7.383772,NaN,8.750809,12.213992,16.269697,10.627451,14.220065,9.401361,17.579167,10.533333,10.540650,NaN,NaN,29.227891,11.637387,15.320513,23.751852,11.983333,14.132143,NaN,NaN,20.764012,NaN,NaN,15.326531,26.301887,NaN,16.358333,NaN,23.0

In [886]:
merge.loc[268]

Year                                                           2015
name_mask                          3f5375683d86822b963caf165a7bbf8c
gender                                                         Male
category                                         Senior Men (23-39)
time                                                        2562.85
Chamonix to Delevret                                        8.02108
Chamonix to La Charme                                           NaN
Chamonix to St-Gervais                                          NaN
Delevret to St-Gervais                                      5.90614
La Charme to St-Gervais                                         NaN
St-Gervais to Contamines                                    8.74531
Contamines to La Balme                                       11.165
La Balme to Bonhomme                                        15.7061
Bonhomme to Chapieux                                        8.15098
Chapieux to Col Seigne                          

In [887]:
merge.groupby('category').count()

,Year,name_mask,gender,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Masters Men 1 (40-49),421,421,421,421,421,0,0,421,0,419,420,421,421,421,421,419,421,421,136,71,350,418,421,421,421,421,70,89,262,70,89,262,367,54,367,54,277,89,54,278,89,54,332,421,421,421,419,0,0,419,0,416,421,421,420,420,421,420,420,421,102,53,368,418,420,420,421,421,108,46,267,108,46,266,342,78,343,78,297,46,77,297,46,78,375,421
Masters Men 2 (50-59),176,176,176,176,176,0,0,176,0,176,176,176,176,176,176,175,176,175,71,33,142,176,176,176,176,176,32,20,123,32,20,124,144,32,144,32,124,20,32,123,20,32,156,176,176,176,175,0,0,176,0,172,176,175,174,176,176,176,176,176,30,22,153,176,176,176,176,176,48,42,86,48,42,85,154,22,154,22,112,42,22,112,42,22,134,176
Masters Men 3 (60-69),15,15,15,15,15,0,0,14,0,15,15,15,15,15,15,15,15,15,6,3,11,15,15,15,15,15,1,0,14,1,0,14,11,4,11,4,11,0,4,11,0,4,15,15,15,15,15,0,0,15,0,15,15,15,15,15,15,15,15,15,0,0,15,15,15,15,15,15,5,8,2,5,8,2,14,1,14,1,6,8,1,6,8,1,7,15
Masters Women 1 (40-49),34,34,34,34,34,0,0,34,0,34,34,34,34,34,34,33,34,34,16,9,24,34,34,34,34,34,4,3,27,4,3,27,31,3,31,3,28,3,3,28,3,3,31,34,34,34,34,0,0,34,0,34,34,34,34,34,34,34,34,34,10,3,31,34,34,34,34,34,6,1,27,6,1,27,24,10,24,10,23,1,10,23,1,10,33,34
Masters Women 2 (50-59),11,11,11,11,11,0,0,11,0,11,11,11,11,11,11,11,11,11,4,1,10,11,11,11,11,11,2,2,7,2,2,7,9,2,9,2,7,2,2,7,2,2,9,11,11,11,11,0,0,11,0,11,11,11,11,11,11,11,11,11,0,0,11,11,11,11,11,11,4,2,5,4,2,5,10,1,10,1,8,2,1,8,2,1,9,11
Masters Women 3 (60-69),2,2,2,2,2,0,0,2,0,2,2,2,2,2,2,2,2,2,1,1,1,2,2,2,2,2,0,0,2,0,0,2,2,0,2,0,2,0,0,2,0,0,2,2,2,2,2,0,0,2,0,2,2,2,2,2,2,2,2,2,0,0,2,2,2,2,2,2,2,0,0,2,0,0,2,0,2,0,2,0,0,2,0,0,2,2
Senior Men (23-39),303,303,303,303,301,0,0,302,0,299,303,303,302,302,302,303,302,301,111,55,247,301,302,303,302,302,64,50,188,64,49,189,283,20,282,20,233,50,20,233,50,20,253,303,303,303,303,0,0,303,0,303,302,302,302,303,303,303,303,302,96,56,245,301,303,301,303,303,48,18,236,48,18,237,216,87,216,87,198,18,87,198,18,87,285,303
Senior Women (23-39),41,41,41,41,41,0,0,41

In [888]:
#merge = merge[merge.category != 'Youth Men (20-22)']
#merge = merge[merge.category != 'Masters Women 3 (60-69)']
#merge = merge[merge.category != 'Masters Men 4 (70-79)']

In [889]:
#df = df[~df["AVG_Pace"] > 12 & df.category == 'Masters Men 3 (60-69)']

merge[(merge["AVG_Pace"] <12) & (merge.category == 'Masters Women 2 (50-59)')]



#df[(df["AVG_Pace"] <12) & (df.category == 'Masters Women 2 (50-59)')]


,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
225,2016,36da23ac15e20c3b3e35b879229da995,Female,Masters Women 2 (50-59),1753.133333,7.149755,NaN,NaN,5.044298,NaN,7.963754,9.11358,12.948485,6.572549,9.780906,14.108163,12.458333,6.766667,8.139024,17.255556,NaN,13.958503,8.116667,7.850641,15.111111,6.043333,7.825,NaN,NaN,10.720649,NaN,NaN,7.764626,15.806918,NaN,7.394444,NaN,16.340693,NaN,NaN,13.543434,NaN,NaN,7.697748,10.218994,Female,1667.866667,7.545343,NaN,NaN,6.116228,NaN,8.108414,9.133745,14.109091,7.745098,10.985437,6.323129,12.966667,7.181481,8.353659,NaN,NaN,16.510204,9.153153,8.86859,16.151852,7.035,8.82619,NaN,NaN,11.446903,NaN,NaN,7.55102,15.613208,NaN,7.3,NaN,15.718615,NaN,NaN,11.636364,NaN,NaN,9.177928,10.148222


In [890]:
merge = merge.drop([30])
merge = merge.drop([347])

In [891]:
merge = merge[merge.name_mask != '4a40c92f6b52512ac605ea7bbcec06b6']
merge = merge[merge.name_mask != '4d5f9076cc4be3cdf23a51be2c1e7d8c']
merge = merge[merge.name_mask != '738138c60e7b32635a37f6d0757c1512']
merge = merge[merge.name_mask != '972253ab3ee99e5c6bcee7a8eba23ebf']
merge = merge[merge.name_mask != 'fbac5ac0b66892696a502c9f3de3ee0e']
merge = merge[merge.name_mask != '083dcbd07948bdbffc8c8a4aeadfef9e']
merge = merge[merge.name_mask != '0b564eb38d30f35edde36b13f5c87cab']
merge = merge[merge.name_mask != '172e495fa981e26fc924feea338a7311']
merge = merge[merge.name_mask != '660743207e40bdedf72dadbe11019157']
merge = merge[merge.name_mask != '804705c96133a4623bf5d9b4144eb932']
merge = merge[merge.name_mask != '831da7840a5dca478f3e2e32426f6b46']
merge = merge[merge.name_mask != 'b0ec1df80b6ee8990cd643ac25f8f606']
merge = merge[merge.name_mask != 'dbc03d5155dc52f0a43dc13b60a93e8d']
merge = merge[merge.name_mask != '2ae85673eb8e4b8fc082e85b200d5064']
merge = merge[merge.name_mask != '38d0ac68447c0ac31140af38b84ee397']
merge = merge[merge.name_mask != '36da23ac15e20c3b3e35b879229da995']
merge = merge[merge.name_mask != '3c3a924c0b32b8eda3e90af5dfd69ce7']
merge = merge[merge.name_mask != 'c10477f93ccf5878840d9fe8b3d1291e']

In [892]:
merge.loc[[767]]

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
767,2015,bdc132ba738a2c8d9989b2b115406d47,Male,Senior Men (23-39),2398.0,7.840931,NaN,NaN,6.555263,NaN,8.890939,10.856379,14.6,8.97451,13.127184,18.247619,18.2875,9.12963,11.760976,72.866667,17.533333,NaN,10.731532,12.08141,20.47037,9.548333,13.958333,NaN,NaN,23.918289,NaN,NaN,13.597959,22.869811,NaN,14.308333,NaN,22.477056,NaN,NaN,14.821212,NaN,NaN,9.634685,16.28353,Male,2370.233333,7.326225,NaN,NaN,5.482895,NaN,8.208091,11.825514,14.930303,8.628105,13.856958,18.162585,16.470833,8.662963,10.590244,79.366667,NaN,16.128571,12.756306,12.738462,19.622222,10.665,11.547619,NaN,NaN,19.233923,NaN,NaN,14.210204,20.483019,NaN,11.705556,NaN,20.622078,NaN,NaN,12.851515,NaN,NaN,9.722523,15.831935


In [893]:
npb[npb.AVG_Pace.isnull()]

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace


In [894]:
merge.iloc[[268]]

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
274,2014,404e6cf3aa6ebe70aa14fb1b2b24a46c,Male,Senior Men (23-39),1761.666667,6.938725,NaN,NaN,5.333333,NaN,7.430421,9.393004,12.790909,7.30719,10.423948,5.887755,13.420833,7.914815,9.284553,NaN,NaN,15.97619,9.68018,10.048077,17.733333,8.005,9.661905,NaN,NaN,12.834808,NaN,NaN,9.894558,18.660377,NaN,9.791667,NaN,17.452381,NaN,NaN,13.580808,NaN,NaN,10.175676,10.817519,Male,1611.116667,6.32598,NaN,NaN,5.469298,NaN,6.917476,8.541152,12.506061,7.071895,10.953074,5.714286,13.029167,7.196296,7.939024,NaN,NaN,16.030612,9.015766,9.358974,16.181481,7.153333,8.635714,NaN,NaN,13.70354,NaN,NaN,9.846939,NaN,15.107639,NaN,12.58046,NaN,NaN,13.40991,NaN,NaN,18.002381,10.934685,10.484381


In [895]:
merge = merge.reset_index(drop=True)

In [896]:
cats = merge['category']
gen = merge['gender']
comb = [cats, gen]
comb = pd.concat(comb, axis=1)

In [897]:
# Update pb and npb to use merged indices
# pb does not contain name_mask, category or year
pb = merge.loc[:, merge.columns.str.contains('pb')]
npb = merge.loc[:, ~merge.columns.str.contains('pb')]

In [898]:
merge.head()

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2312.483333,9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,14.183180,Male,2183.416667,9.475000,NaN,NaN,6.922368,NaN,10.046602,11.295473,16.930303,8.332026,13.920388,7.668027,17.250000,8.637037,10.173171,NaN,NaN,22.579592,10.728378,11.035897,23.777778,7.825000,12.182143,13.344108,NaN,NaN,12.647619,NaN,NaN,19.703774,NaN,15.233333,NaN,22.316450,NaN,NaN,18.892929,NaN,NaN,15.230631,13.589501
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2581.233333,10.594853,NaN,NaN,10.560088,NaN,11.344984,14.690123,18.672727,10.256209,14.532686,9.376190,17.733333,10.933333,13.152033,NaN,NaN,25.886395,11.713063,16.156410,25.733333,15.486667,17.373810,NaN,20.975426,NaN,NaN,16.459829,NaN,21.488050,NaN,11.588889,NaN,NaN,10.667196,NaN,NaN,11.662452,NaN,NaN,15.088612,Male,2183.416667,9.475000,NaN,NaN,6.922368,NaN,10.046602,11.295473,16.930303,8.332026,13.920388,7.668027,17.250000,8.637037,10.173171,NaN,NaN,22.579592,10.728378,11.035897,23.777778,7.825000,12.182143,13.344108,NaN,NaN,12.647619,NaN,NaN,19.703774,NaN,15.233333,NaN,22.316450,NaN,NaN,18.892929,NaN,NaN,15.230631,13.589501
2,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),2703.116667,8.112990,NaN,NaN,6.386404,NaN,9.622330,13.158848,18.600000,10.487582,15.763107,25.271429,15.537500,9.570370,10.208130,47.2,NaN,21.417687,13.706757,16.427564,20.859259,10.913333,13.300000,NaN,NaN,21.288496,NaN,NaN,14.502721,24.690566,NaN,12.572222,NaN,25.139394,NaN,NaN,20.159596,NaN,NaN,15.157207,16.802140,Male,2456.833333,8.466912,NaN,NaN,7.383772,NaN,8.750809,12.213992,16.269697,10.627451,14.220065,9.401361,17.579167,10.533333,10.540650,NaN,NaN,29.227891,11.637387,15.320513,23.751852,11.983333,14.132143,NaN,NaN,20.764012,NaN,NaN,15.326531,26.301887,NaN,16.358333,NaN,23.0

In [899]:
merge[merge.category == 'Masters Women 2 (50-59)']

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
71,2015,1320ec5e4c7a443d78f71103526fc704,Female,Masters Women 2 (50-59),2622.516667,10.344608,NaN,NaN,8.864474,NaN,11.208091,12.611523,20.160606,10.958170,15.255016,23.710204,24.425000,10.885185,18.415447,10.292593,27.9,NaN,14.551351,13.151923,26.685185,10.731667,15.326190,NaN,NaN,18.964012,NaN,NaN,13.220408,20.316352,NaN,10.250000,NaN,20.390476,NaN,NaN,16.856566,NaN,NaN,11.371171,15.873849,Female,2495.433333,9.474265,NaN,NaN,8.260965,NaN,10.428803,13.345679,19.806061,10.506536,16.721683,9.571429,19.850000,10.966667,13.044715,NaN,NaN,28.846939,13.227477,15.067308,24.859259,11.116667,13.071429,NaN,NaN,18.762537,NaN,NaN,18.234694,NaN,21.086806,NaN,24.100575,NaN,NaN,18.310811,NaN,NaN,23.642857,14.745495,16.127069
72,2013,13213e63d3265f0750495f08ab12d901,Female,Masters Women 2 (50-59),2358.333333,9.615931,NaN,NaN,8.836842,NaN,11.022330,12.003292,18.512121,11.201307,13.943042,10.623810,16.650000,9.981481,12.018699,NaN,NaN,23.375510,13.822973,14.801923,21.414815,11.560000,13.553571,17.131987,NaN,NaN,16.949206,NaN,NaN,24.449057,NaN,16.075000,NaN,18.574026,NaN,NaN,13.716162,NaN,NaN,8.771171,14.525177,Female,2264.033333,9.509069,NaN,NaN,9.285965,NaN,9.563430,11.891770,16.054545,9.883660,12.660518,8.308163,15.045833,9.440741,11.021951,NaN,NaN,20.934014,10.722072,11.153205,18.318519,12.185000,13.039286,NaN,17.122628,NaN,NaN,18.801709,NaN,21.761635,NaN,13.283333,NaN,NaN,12.524339,NaN,NaN,12.945977,NaN,NaN,13.280755
188,2016,30f977102cb8714bf2b596874ef040d2,Female,Masters Women 2 (50-59),2670.166667,8.772304,NaN,NaN,7.632018,NaN,10.636893,15.644444,20.039394,9.405882,17.313269,20.434694,20.237500,9.281481,12.301626,54.681481,NaN,19.536735,11.805856,11.946795,23.014815,9.913333,15.080952,NaN,NaN,21.546608,NaN,NaN,15.655782,24.898113,NaN,10.541667,NaN,21.100433,NaN,NaN,16.129293,NaN,NaN,9.672973,16.688974,Female,2044.233333,9.056863,NaN,NaN,7.413158,NaN,8.849838,12.618107,17.106061,7.324837,13.723625,6.794558,13.216667,10.448148,8.334959,NaN,NaN,20.961224,10.348198,9.973718,19.025926,9.723333,11.217857,NaN,14.171290,NaN,NaN,17.810256,Na

In [900]:
merge.head()

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2312.483333,9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,14.183180,Male,2183.416667,9.475000,NaN,NaN,6.922368,NaN,10.046602,11.295473,16.930303,8.332026,13.920388,7.668027,17.250000,8.637037,10.173171,NaN,NaN,22.579592,10.728378,11.035897,23.777778,7.825000,12.182143,13.344108,NaN,NaN,12.647619,NaN,NaN,19.703774,NaN,15.233333,NaN,22.316450,NaN,NaN,18.892929,NaN,NaN,15.230631,13.589501
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2581.233333,10.594853,NaN,NaN,10.560088,NaN,11.344984,14.690123,18.672727,10.256209,14.532686,9.376190,17.733333,10.933333,13.152033,NaN,NaN,25.886395,11.713063,16.156410,25.733333,15.486667,17.373810,NaN,20.975426,NaN,NaN,16.459829,NaN,21.488050,NaN,11.588889,NaN,NaN,10.667196,NaN,NaN,11.662452,NaN,NaN,15.088612,Male,2183.416667,9.475000,NaN,NaN,6.922368,NaN,10.046602,11.295473,16.930303,8.332026,13.920388,7.668027,17.250000,8.637037,10.173171,NaN,NaN,22.579592,10.728378,11.035897,23.777778,7.825000,12.182143,13.344108,NaN,NaN,12.647619,NaN,NaN,19.703774,NaN,15.233333,NaN,22.316450,NaN,NaN,18.892929,NaN,NaN,15.230631,13.589501
2,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),2703.116667,8.112990,NaN,NaN,6.386404,NaN,9.622330,13.158848,18.600000,10.487582,15.763107,25.271429,15.537500,9.570370,10.208130,47.2,NaN,21.417687,13.706757,16.427564,20.859259,10.913333,13.300000,NaN,NaN,21.288496,NaN,NaN,14.502721,24.690566,NaN,12.572222,NaN,25.139394,NaN,NaN,20.159596,NaN,NaN,15.157207,16.802140,Male,2456.833333,8.466912,NaN,NaN,7.383772,NaN,8.750809,12.213992,16.269697,10.627451,14.220065,9.401361,17.579167,10.533333,10.540650,NaN,NaN,29.227891,11.637387,15.320513,23.751852,11.983333,14.132143,NaN,NaN,20.764012,NaN,NaN,15.326531,26.301887,NaN,16.358333,NaN,23.0

In [901]:
pb.head()

,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
0,Male,2183.416667,9.475000,NaN,NaN,6.922368,NaN,10.046602,11.295473,16.930303,8.332026,13.920388,7.668027,17.250000,8.637037,10.173171,NaN,NaN,22.579592,10.728378,11.035897,23.777778,7.825000,12.182143,13.344108,NaN,NaN,12.647619,NaN,NaN,19.703774,NaN,15.233333,NaN,22.316450,NaN,NaN,18.892929,NaN,NaN,15.230631,13.589501
1,Male,2183.416667,9.475000,NaN,NaN,6.922368,NaN,10.046602,11.295473,16.930303,8.332026,13.920388,7.668027,17.250000,8.637037,10.173171,NaN,NaN,22.579592,10.728378,11.035897,23.777778,7.825000,12.182143,13.344108,NaN,NaN,12.647619,NaN,NaN,19.703774,NaN,15.233333,NaN,22.316450,NaN,NaN,18.892929,NaN,NaN,15.230631,13.589501
2,Male,2456.833333,8.466912,NaN,NaN,7.383772,NaN,8.750809,12.213992,16.269697,10.627451,14.220065,9.401361,17.579167,10.533333,10.540650,NaN,NaN,29.227891,11.637387,15.320513,23.751852,11.983333,14.132143,NaN,NaN,20.764012,NaN,NaN,15.326531,26.301887,NaN,16.358333,NaN,23.008658,NaN,NaN,18.101010,NaN,NaN,11.027027,15.121991
3,Male,2594.800000,8.111765,NaN,NaN,7.417105,NaN,10.208091,12.413992,19.266667,11.023529,14.892557,23.220408,18.791667,11.070370,11.923577,29.033333,27.475000,NaN,15.308108,16.235256,24.625926,10.751667,14.061905,NaN,NaN,20.912389,NaN,NaN,12.740816,24.894969,NaN,12.330556,NaN,21.654545,NaN,NaN,17.629293,NaN,NaN,12.161712,16.326208
4,Male,2691.966667,7.839706,NaN,NaN,6.164912,NaN,9.394175,12.763786,17.118182,9.997386,16.763107,24.523129,22.962500,11.662963,15.659350,60.033333,24.970833,NaN,14.623423,15.655128,23.637037,11.198333,15.998810,NaN,NaN,21.316519,NaN,NaN,13.346259,28.514465,NaN,11.900000,NaN,21.643723,NaN,NaN,17.775758,NaN,NaN,13.675225,17.965522


In [902]:
pb.to_csv('../../data/pb_times.csv')

In [903]:
npb.head()

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2312.483333,9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,14.183180
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2581.233333,10.594853,NaN,NaN,10.560088,NaN,11.344984,14.690123,18.672727,10.256209,14.532686,9.376190,17.733333,10.933333,13.152033,NaN,NaN,25.886395,11.713063,16.156410,25.733333,15.486667,17.373810,NaN,20.975426,NaN,NaN,16.459829,NaN,21.488050,NaN,11.588889,NaN,NaN,10.667196,NaN,NaN,11.662452,NaN,NaN,15.088612
2,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),2703.116667,8.112990,NaN,NaN,6.386404,NaN,9.622330,13.158848,18.600000,10.487582,15.763107,25.271429,15.537500,9.570370,10.208130,47.2,NaN,21.417687,13.706757,16.427564,20.859259,10.913333,13.300000,NaN,NaN,21.288496,NaN,NaN,14.502721,24.690566,NaN,12.572222,NaN,25.139394,NaN,NaN,20.159596,NaN,NaN,15.157207,16.802140
3,2013,011b597a8c4121461b659f50dbd19d5a,Male,Masters Men 1 (40-49),2602.466667,9.005637,NaN,NaN,8.174561,NaN,11.711650,11.610288,21.154545,11.524837,16.530421,11.491156,23.391667,11.670370,12.002439,NaN,NaN,26.926531,12.949099,13.702564,23.129630,12.508333,12.977381,18.426599,NaN,NaN,19.700529,NaN,NaN,29.606289,NaN,16.152778,NaN,25.409524,NaN,NaN,15.660606,NaN,NaN,14.259910,16.236556
4,2017,01368b79f758adce8d638a35b80c8a99,Male,Senior Men (23-39),2699.366667,9.366422,NaN,NaN,8.320175,NaN,10.805825,12.833333,19.327273,13.058824,16.778317,11.306122,23.329167,12.470370,15.512195,NaN,NaN,35.234694,15.292793,15.272436,27.085185,11.908333,18.986905,NaN,NaN,22.383481,NaN,NaN,18.176871,NaN,21.732639,NaN,19.284483,NaN,NaN,21.981982,NaN,NaN,19.604762,15.479730,17.313847


# Define Functions

In [904]:
def find_similar_times(index):
    query = npb.iloc[index]
    same_category = npb[npb.category == query.category]
    similar_times = same_category[(same_category.AVG_Pace > query.AVG_Pace - pace_threshold) & (same_category.AVG_Pace < query.AVG_Pace + pace_threshold)]
    #find all runners either 0.5 minutes/kilometre faster or slower than the query runner
   # if len(similar_times) = 1:
    #    similar_times
    return similar_times
    #else:
     #   return None

# Show sample output
find_similar_times(30)

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace
17,2015,04578c529d977b6912c3a413df104431,Male,Masters Men 1 (40-49),1763.533333,7.284559,NaN,NaN,5.274561,NaN,7.390939,9.058025,13.115152,7.059477,10.949191,15.529932,14.537500,6.933333,8.240650,42.644444,16.554167,NaN,8.573874,9.334615,16.781481,7.923333,10.617857,NaN,NaN,13.062832,NaN,NaN,7.887075,14.703145,NaN,6.333333,NaN,13.405628,NaN,NaN,9.992929,NaN,NaN,6.175225,11.574530
30,2011,07dd98de0ea64ae2dfc7e2cfac8a1ead,Male,Masters Men 1 (40-49),1976.133333,7.523775,NaN,NaN,6.500877,NaN,7.152427,8.369136,14.081818,8.445752,10.959871,6.087075,13.812500,6.970370,8.115447,NaN,NaN,20.335374,9.807658,14.005769,26.203704,11.921667,10.590476,NaN,14.496107,NaN,NaN,19.611538,NaN,20.252201,NaN,9.163889,NaN,NaN,9.193651,NaN,NaN,9.250575,NaN,NaN,11.863115
33,2011,0999c52336f41c6ee35be9fb5f01fbbd,Male,Masters Men 1 (40-49),1968.116667,7.574020,NaN,NaN,6.400000,NaN,7.726861,9.900000,16.312121,8.079739,12.537540,7.202721,15.025000,7.977778,9.225203,NaN,NaN,23.828571,10.064414,11.098718,18.740741,9.830000,11.195238,NaN,13.672506,NaN,NaN,15.761111,NaN,18.450314,NaN,10.430556,NaN,NaN,11.394709,NaN,NaN,9.162452,NaN,NaN,11.808275
35,2013,09d9cc6f4f2b2669695eb7de6941ac6b,Male,Masters Men 1 (40-49),2002.683333,7.928431,NaN,NaN,5.602193,NaN,8.020712,9.433333,13.881818,7.554248,12.467314,7.409524,14.570833,8.629630,9.465854,NaN,NaN,21.780272,9.543694,10.192949,17.729630,7.823333,11.077381,15.652189,NaN,NaN,13.203175,NaN,NaN,20.244654,NaN,9.713889,NaN,21.896537,NaN,NaN,14.423232,NaN,NaN,17.737387,12.332592
44,2011,0bf37bc01ce5b07244096b7f336ee496,Male,Masters Men 1 (40-49),2026.716667,7.244363,NaN,NaN,6.421930,NaN,7.391909,10.151029,14.700000,8.576471,12.969579,7.025850,17.920833,8.207407,10.078862,NaN,NaN,23.362585,10.476577,12.800641,19.596296,8.920000,10.482143,NaN,14.240633,NaN,NaN,17.641453,NaN,21.003774,NaN,12.466667,NaN,NaN,10.675132,NaN,NaN,10.647126,NaN,NaN,12.304403
67,2013,11a8b5049ddaf0d2782483bf36322b0f,Male,Masters Men 1 (40-49),2017.733333,8.131863,NaN,NaN,6.325877,NaN,9.802265,10.620576,15.669697,8.822222,13.755340,8.804082,18.979167,10.796296,11.299187,NaN,NaN,27.691837,10.793694,11.314744,18.811111,8.136667,10.598810,13.589899,NaN,NaN,12.203175,NaN,NaN,19.782390,NaN,9.750000,NaN,18.275325,NaN,NaN,10.756566,NaN,NaN,8.426577,12.630723
68,2014,12ba514c1c91d3f6bc78d38c97f858ec,Male,Masters Men 1 (40-49),1840.816667,5.773284,NaN,NaN,4.967105,NaN,6.681230,8.695473,13.318182,6.728758,10.457929,8.149660,18.487500,11.218519,14.552846,NaN,NaN,16.993197,11.085586,10.221154,20.785185,9.235000,11.191667,NaN,NaN,14.445428,NaN,NaN,9.027211,15.996855,NaN,9.047222,NaN,15.874459,NaN,NaN,15.191919,NaN,NaN,8.927928,11.543887
94,2015,17ddb5c4452fb4199f1ad1d21d44f181,Male,Masters Men 1 (40-49),2004.583333,8.239216,NaN,NaN,5.743860,NaN,8.568932,10.574486,14.054545,7.059477,12.120712,15.958503,16.366667,7.507407,9.041463,36.922222,17.154167,NaN,8.792342,10.437179,18.588889,7.298333,11.764286,NaN,NaN,14.894690,NaN,NaN,8.757823,21.684277,NaN,9.511111,NaN,19.024675,NaN,NaN,13.866667,NaN,NaN,7.249550,12.847259
113,2011,1bff26b92cf956ec02cb198b463e144a,Male,M

In [908]:
def cosine_similarity(a, b):
    # Combine two lists (row and query run) into matrix
    matrix = pd.DataFrame({"A": a, "B": b})
    # Now drop any pairs containing NA so we only compare when we have two times
    matrix = matrix.dropna(axis = 0, how='any')
    # Parse back to two lists
    a = matrix[['A']]
    b = matrix[['B']]
    # Return cosine similarity (1 - cosine difference)
    return 1 - (cosine(a, b))

def sims(index):
    similar_times = find_similar_times(index)
    #similar_times = similar_times.dropna(how='all')
    #if sim_count > 1:
    similar_times = similar_times.filter(like='to')
    #if similar_times.count() > 1:
    query_segs = list(similar_times.loc[index])
    similar_times = similar_times.drop([index], axis = 0)
    result = similar_times.apply(lambda row: cosine_similarity(list(row), query_segs), axis = 1)
    result = result.sort_values(ascending = False) #finding the most similar
    result = result.head(10) #10 is the number of neighbours as an example, the optimal value for n is tested in Prediction_Eval notebook
    
    return result

    #elif sim_count <= 1:
     #   return 0

sims(30)

993    0.988886
589    0.988698
398    0.987643
445    0.987033
478    0.986779
251    0.986748
435    0.986040
237    0.985641
44     0.985547
970    0.984767
dtype: float64

In [909]:
def euc_distance(a, b):
    # Combine two lists (row and query run) into matrix
    matrix = pd.DataFrame({"A": a, "B": b})
    # Now drop any pairs containing NA so we only compare when we have two times
    matrix = matrix.dropna(axis = 0, how='any')
    # Parse back to two lists
    a = matrix[['A']]
    b = matrix[['B']]
    
    return euclidean(a,b)

def euc_sims(index):
    similar_times = find_similar_times(index)
    #similar_times = similar_times.dropna(how='all')
    #if sim_count > 1:
    similar_times = similar_times.filter(like='to')
    #if similar_times.count() > 1:
    query_segs = list(similar_times.loc[index])
    similar_times = similar_times.drop([index], axis = 0)
    result = similar_times.apply(lambda row: euc_distance(list(row), query_segs), axis = 1)
    result = result.sort_values(ascending = True) #finding the most similar
    result = result.head(10) #10 is the number of neighbours as an example, the optimal value for n is tested in Prediction_Eval notebook
    
    return result

    #elif sim_count <= 1:
     #   return 0

euc_sims(30)

445     9.000827
589     9.330050
478     9.497396
993     9.755274
398    10.018417
816    10.073437
444    10.089975
970    10.173764
251    10.225070
435    10.366421
dtype: float64

In [910]:
def weight(index):
    weighter = sims(index)
    #weight_count = len(weighter)
    #weight0 = len(str(weight_count))
   # if weight_count > 1:
   # if weighter.sum() > 3:
    if len(weighter) > 1:
        weighter = weighter / weighter.sum()
        return weighter
    
    else:
        weighter1 = weighter / weighter
        return weighter1 / 10
    
    #else:
     #   return np.nan
    
    #elif weight0 <= 1:
     #   return 0
    

weight(30)

993    0.100214
589    0.100195
398    0.100088
445    0.100026
478    0.100000
251    0.099997
435    0.099925
237    0.099885
44     0.099875
970    0.099796
dtype: float64

In [911]:
def euc_weight(index):
    weighter = euc_sims(index)
    if len(weighter) > 1:
        weighter = 1 -(weighter / weighter.sum())
        weighter = weighter / weighter.sum()
        return weighter
    else:
        weighter1 = weighter / weighter
        return weighter1 / 10
    
euc_weight(66)

628    0.101507
677    0.101296
389    0.101289
87     0.100076
898    0.099996
822    0.099704
525    0.099250
128    0.099170
135    0.099019
299    0.098692
dtype: float64

In [913]:
def find_nearest_neighbours(index):
    #w = weight(index)
    cosine_distance = weight(index)  
    nearest_neighbours = merge.iloc[cosine_distance.index]
    if len(nearest_neighbours) > 1:
        nearest_neighbours['weights'] = weight(index)
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    else: 
        nearest_neighbours['weights'] = weight(index) * 10
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
        
        return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours(66)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb,weights,weighted_pace
389,2011,60bcdc7d4bbd60dc447ee85f8f44572d,Male,Masters Men 2 (50-59),2713.433333,11.254167,NaN,NaN,10.099561,NaN,11.613592,13.175720,19.218182,11.354248,17.762460,12.148299,21.062500,13.162963,13.107317,NaN,NaN,26.280952,13.544144,15.867949,23.970370,17.396667,16.140476,NaN,19.799027,NaN,NaN,19.175641,NaN,20.962893,NaN,9.266667,NaN,NaN,16.220106,NaN,NaN,14.696935,NaN,NaN,15.968732,Male,2699.616667,10.313235,NaN,NaN,8.735965,NaN,12.392880,13.206996,19.851515,13.384314,16.294175,12.385714,22.158333,12.640741,14.734146,NaN,NaN,26.923129,15.018919,16.241026,24.055556,12.858333,15.696429,18.857576,NaN,NaN,20.623810,NaN,NaN,24.754088,NaN,14.813889,NaN,23.682251,NaN,NaN,21.170707,NaN,NaN,12.205856,16.791649,0.100168,1.681992
822,2011,cc68b5f4c2e2eeb5875d95d7ea6398c9,Male,Masters Men 2 (50-59),2568.800000,9.047059,NaN,NaN,9.049123,NaN,10.071521,13.206584,17.839394,10.334641,14.519741,10.597279,18.820833,10.974074,13.814634,NaN,NaN,24.685714,13.104955,15.758974,25.303704,16.188333,15.798810,NaN,19.316058,NaN,NaN,19.342308,NaN,21.994340,NaN,13.466667,NaN,NaN,13.418519,NaN,NaN,13.949808,NaN,NaN,15.243612,Male,2531.050000,8.330392,NaN,NaN,7.113158,NaN,12.897735,12.562963,19.736364,12.910458,17.616181,12.256463,17.175000,11.100000,12.608130,NaN,NaN,27.018367,13.334234,15.596795,22.522222,10.863333,13.550000,18.675758,NaN,NaN,17.126455,NaN,NaN,24.923899,NaN,15.544444,NaN,23.258009,NaN,NaN,15.761616,NaN,NaN,13.329730,15.658821,0.100061,1.566837
628,2017,9e6c04f5c8d6bc11e02861369b7d3c87,Male,Masters Men 2 (50-59),2517.900000,10.176471,NaN,NaN,9.289474,NaN,10.836570,12.716049,19.669697,11.398693,16.284790,9.363946,19.887500,10.562963,13.658537,NaN,NaN,29.792517,12.878378,16.660256,26.333333,11.616667,13.813095,NaN,NaN,21.212389,NaN,NaN,16.139456,NaN,22.576389,NaN,18.580460,NaN,NaN,18.635135,NaN,NaN,20.547619,15.074324,16.154363,Male,2424.083333,9.626225,NaN,NaN,8.592105,NaN,10.618123,13.845679,18.878788,10.130719,14.689320,9.993197,18.637500,11.840741,13.199187,NaN,NaN,24.904762,12.162162,13.698718,22.796296,11.00500

In [914]:
def euc_find_nearest_neighbours(index):
    #w = weight(index)
    euc_distance = euc_weight(index)  
    nearest_neighbours = merge.iloc[euc_distance.index]
    if len(nearest_neighbours) > 1: 
        nearest_neighbours['weights'] = euc_weight(index)
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    
    else: 
        nearest_neighbours['weights'] = euc_weight(index) * 10
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb'] 
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    
# Show sample output    
euc_find_nearest_neighbours(66)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb,weights,weighted_pace
628,2017,9e6c04f5c8d6bc11e02861369b7d3c87,Male,Masters Men 2 (50-59),2517.900000,10.176471,NaN,NaN,9.289474,NaN,10.836570,12.716049,19.669697,11.398693,16.284790,9.363946,19.887500,10.562963,13.658537,NaN,NaN,29.792517,12.878378,16.660256,26.333333,11.616667,13.813095,NaN,NaN,21.212389,NaN,NaN,16.139456,NaN,22.576389,NaN,18.580460,NaN,NaN,18.635135,NaN,NaN,20.547619,15.074324,16.154363,Male,2424.083333,9.626225,NaN,NaN,8.592105,NaN,10.618123,13.845679,18.878788,10.130719,14.689320,9.993197,18.637500,11.840741,13.199187,NaN,NaN,24.904762,12.162162,13.698718,22.796296,11.005000,14.350000,NaN,NaN,19.271386,NaN,NaN,13.758503,21.591195,NaN,12.633333,NaN,21.062771,NaN,NaN,17.540404,NaN,NaN,10.317568,14.797653,0.101507,1.502064
677,2017,a8891c0512542721258c94c526eadbef,Male,Masters Men 2 (50-59),2609.033333,10.481618,NaN,NaN,9.679825,NaN,11.577670,13.625514,20.663636,12.627451,16.084142,10.459184,19.416667,11.266667,13.674797,NaN,NaN,30.605442,15.018018,15.573718,24.859259,11.671667,15.210714,NaN,NaN,20.656342,NaN,NaN,18.289116,NaN,24.243056,NaN,20.066092,NaN,NaN,20.229730,NaN,NaN,21.652381,13.788288,16.725875,Male,2536.900000,9.501961,NaN,NaN,8.303947,NaN,11.583819,11.655556,18.390909,11.635948,13.994822,10.576190,18.491667,10.637037,12.417073,NaN,NaN,24.072789,12.723874,13.507051,22.288889,12.046667,13.790476,19.566330,NaN,NaN,18.184656,NaN,NaN,25.625157,NaN,17.072222,NaN,32.063203,NaN,NaN,13.807071,NaN,NaN,11.266667,15.550166,0.101296,1.575174
389,2011,60bcdc7d4bbd60dc447ee85f8f44572d,Male,Masters Men 2 (50-59),2713.433333,11.254167,NaN,NaN,10.099561,NaN,11.613592,13.175720,19.218182,11.354248,17.762460,12.148299,21.062500,13.162963,13.107317,NaN,NaN,26.280952,13.544144,15.867949,23.970370,17.396667,16.140476,NaN,19.799027,NaN,NaN,19.175641,NaN,20.962893,NaN,9.266667,NaN,NaN,16.220106,NaN,NaN,14.696935,NaN,NaN,15.968732,Male,2699.616667,10.313235,NaN,NaN,8.735965,NaN,12.392880,13.206996,19.851515,13.384314,16.294175,12.385714,22.158333,12.640741,14.734146,NaN,NaN,26.923129,15.018919,16.241026,24.055556,1

In [915]:
def find_nearest_neighbours_pace(index):
    #w = weight(index)
    cosine_distance = weight(index)  
    nearest_neighbours = merge.iloc[cosine_distance.index]
    #nearest_neighbours = nearest_neighbours.filter(like='to')
    #nearest_neighbours['weights'] = weight(index)
    #nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = nearest_neighbours.loc[:, 'Chamonix to Delevret_pb':'Flegere to Chamonix_pb'].mul(nearest_neighbours['weighted_pace'], axis=0)
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
    return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours_pace(5)

,Year,name_mask,gender,category,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,gender_pb,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
336,2011,53f4783fc98e1be6bd6546e1ff27872d,Male,Senior Men (23-39),2394.900000,8.471078,NaN,NaN,7.676316,NaN,8.599029,12.011111,17.151515,9.187582,13.895146,7.825170,19.520833,8.877778,11.643902,NaN,NaN,29.641497,12.771622,13.031410,22.759259,14.968333,15.446429,NaN,19.712652,NaN,NaN,17.658547,NaN,21.381132,NaN,10.269444,NaN,NaN,10.976720,NaN,NaN,11.750575,NaN,NaN,14.140308,Male,2084.083333,8.593873,NaN,NaN,6.442105,NaN,10.117799,10.721399,15.184848,8.616340,12.947896,7.912925,16.591667,8.929630,11.339837,NaN,NaN,21.290476,10.795946,11.612821,20.837037,9.173333,12.321429,13.911448,NaN,NaN,12.504762,NaN,NaN,22.033962,NaN,10.772222,NaN,20.290476,NaN,NaN,14.887879,NaN,NaN,11.561712,12.891326
935,2011,ed2d4176e498ce0a3a01e3ea0e73148b,Male,Senior Men (23-39),2459.283333,8.168382,NaN,NaN,7.588596,NaN,9.181553,13.972016,16.578788,8.641830,14.445307,8.053061,17.541667,9.396296,13.753659,NaN,NaN,28.842177,11.456306,14.297436,26.618519,14.870000,17.357143,NaN,19.449878,NaN,NaN,16.453419,NaN,19.365409,NaN,11.511111,NaN,NaN,12.791534,NaN,NaN,13.141379,NaN,NaN,14.498933,Male,2359.433333,8.914216,NaN,NaN,7.429825,NaN,9.362460,12.847737,18.000000,9.764706,14.263754,9.292517,18.070833,9.429630,11.195122,NaN,NaN,29.697279,10.903153,12.432692,19.996296,10.691667,12.057143,NaN,NaN,19.393805,NaN,NaN,13.479592,24.022013,NaN,14.477778,NaN,25.493506,NaN,NaN,17.287879,NaN,NaN,9.470721,14.498930
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),2581.233333,10.594853,NaN,NaN,10.560088,NaN,11.344984,14.690123,18.672727,10.256209,14.532686,9.376190,17.733333,10.933333,13.152033,NaN,NaN,25.886395,11.713063,16.156410,25.733333,15.486667,17.373810,NaN,20.975426,NaN,NaN,16.459829,NaN,21.488050,NaN,11.588889,NaN,NaN,10.667196,NaN,NaN,11.662452,NaN,NaN,15.088612,Male,2183.416667,9.475000,NaN,NaN,6.922368,NaN,10.046602,11.295473,16.930303,8.332026,13.920388,7.668027,17.250000,8.637037,10.173171,NaN,NaN,22.579592,10.728378,11.035897,23.777778,7.825000,12.182143,13.344108,NaN,NaN,12.647619,NaN,NaN,19.703774,NaN,15.233333,NaN,22.316450,NaN,NaN,18

In [916]:
def euc_find_nearest_neighbours_pace(index):
    #w = weight(index)
    euc_distance = euc_weight(index)  
    nearest_neighbours = merge.iloc[euc_distance.index]
    if len(nearest_neighbours) > 1:
        nearest_neighbours['weights'] = euc_weight(index)
        #nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
        nearest_neighbours = nearest_neighbours.loc[:, 'Chamonix to Delevret_pb':'Flegere to Chamonix_pb'].mul(nearest_neighbours['weights'], axis=0)
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    
    
    
# Show sample output    
euc_find_nearest_neighbours_pace(66)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb
628,0.977129,NaN,NaN,0.872158,NaN,1.077813,1.405432,1.916328,1.028338,1.491068,1.014379,1.891836,1.201917,1.339809,NaN,NaN,2.528006,1.234544,1.390515,2.313982,1.117084,1.456625,NaN,NaN,1.956179,NaN,NaN,1.396584,2.191656,NaN,1.282371,NaN,2.138017,NaN,NaN,1.780473,NaN,NaN,1.047305
677,0.962514,NaN,NaN,0.841159,NaN,1.173398,1.180665,1.862931,1.178679,1.417624,1.071329,1.873138,1.077493,1.257804,NaN,NaN,2.438485,1.288882,1.368215,2.257782,1.220283,1.396924,1.981997,NaN,NaN,1.842039,NaN,NaN,2.595734,NaN,1.729353,NaN,3.247884,NaN,NaN,1.398605,NaN,NaN,1.141272
389,1.044616,NaN,NaN,0.884856,NaN,1.255261,1.337721,2.010737,1.355682,1.650418,1.254535,2.244392,1.280366,1.492405,NaN,NaN,2.727013,1.521249,1.645035,2.436560,1.302406,1.589873,1.910062,NaN,NaN,2.088962,NaN,NaN,2.507313,NaN,1.500482,NaN,2.398748,NaN,NaN,2.144357,NaN,NaN,1.236317
87,0.825382,NaN,NaN,0.739597,NaN,0.935662,1.220062,1.742839,1.114245,1.349893,0.995654,1.799283,1.063400,1.253798,NaN,NaN,2.241498,1.187614,1.230101,2.042663,1.138865,1.261196,NaN,NaN,1.668671,NaN,NaN,1.483440,2.280034,NaN,1.464168,NaN,2.407456,NaN,NaN,2.101091,NaN,NaN,1.251626
898,1.116572,NaN,NaN,1.013467,NaN,1.204287,1.456442,2.134458,1.474972,1.719314,1.107233,2.030333,1.231431,1.336774,NaN,NaN,2.422485,1.488317,1.390391,2.517674,1.242615,1.608029,1.954196,NaN,NaN,1.947221,NaN,NaN,2.825921,NaN,1.408831,NaN,2.078962,NaN,NaN,1.199850,NaN,NaN,1.426833
822,0.830570,NaN,NaN,0.709208,NaN,1.285951,1.252573,1.967787,1.287219,1.756397,1.222014,1.712410,1.106710,1.257076,NaN,NaN,2.693829,1.329471,1.555057,2.245547,1.083114,1.350984,1.862040,NaN,NaN,1.707569,NaN,NaN,2.485003,NaN,1.549837,NaN,2.318907,NaN,NaN,1.571490,NaN,NaN,1.329022
525,0.811591,NaN,NaN,0.645911,NaN,1.129142,1.121489,1.631616,0.905125,1.460458,0.886974,1.920081,1.041026,1.188423,NaN,NaN,2.727833,1.385571,1.307178,2.126899,0.957105,1.240630,1.508205,NaN,NaN,1.404419,NaN,NaN,2.361036,NaN,1.070526,NaN,1.906639,NaN,NaN,1.291659,NaN,NaN,1.084690
128,0.976440,NaN,NaN,0.904800,NaN,1.009549,1.276034,1.819627,0.940628,1.456457,0.955139,0.597502,3.340942,1.718309,NaN,NaN,2.651831,1.379362,1.331109,2.284593,1.422765,1.448124,NaN,1.652937,NaN,NaN,1.686999,NaN,2.021518,NaN,1.151754,NaN,NaN,1.230500,NaN,NaN,1.279299,NaN,NaN
135,0.976649,NaN,NaN,0.919271,NaN,0.994069,1.274698,1.818051,1.015561,1.414660,1.018011,1.884664,1.042635,1.723738,NaN,NaN,2.229817,1.336446,1.434762,2.674250,1.473405,1.408429,NaN,1.651018,NaN,NaN,1.691831,NaN,2.005355,NaN,1.150822,NaN,NaN,1.228361,NaN,NaN,1.277537,NaN,NaN
299,0.809278,NaN,NaN,0.663837,NaN,0.876191,1.303146,1.646070,0.916034,1.525198,1.819701,1.745623,0.801968,1.028728,7.019592,NaN,1.628358,1.160036,1.345444,2.251284,0.924090,1.167861,NaN,NaN,1.768836,NaN,NaN,1.283270,2.141316,NaN,1.158814,NaN,2.082453,NaN,NaN,1.446293,NaN,NaN,0.767712


In [917]:
def weighted_row(neighbours):
    #row_pace = find_nearest_neighbours_pace(index)
    row_pace = neighbours.mean()
    row_pace = pd.DataFrame(row_pace)
    row_pace = row_pace.T
    #pred_pace = row_pace
    return row_pace

neighbours = find_nearest_neighbours_pace(66)
weighted_row(neighbours)

,Year,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
0,2014.3,2546.64,9.674338,NaN,NaN,9.083026,NaN,10.81,13.582428,18.837273,11.243007,15.810971,10.139932,20.07,11.030741,13.125447,NaN,NaN,27.737551,13.35527,15.485962,24.84037,13.680167,14.520714,NaN,18.696837,20.907375,NaN,17.901068,24.422336,21.916855,22.002778,11.713333,20.636207,23.428571,13.704894,20.20991,14.010101,14.823851,21.418095,13.434309,15.95194,2414.076667,9.24027,NaN,NaN,8.004518,NaN,11.556778,13.785514,18.398788,11.408366,15.008544,10.08483,18.978333,11.058519,12.748211,NaN,NaN,24.68585,12.965405,13.349103,21.984444,10.9335,13.551548,17.571942,NaN,18.742625,16.91261,NaN,13.666667,23.36566,NaN,13.348611,NaN,22.269524,NaN,NaN,16.147879,NaN,NaN,11.456847,14.920343


In [918]:
def pace_rec(index):
    #print(index)
    query = npb.iloc[index]
    query = pd.DataFrame(query)
    query = query.T
    row_pace = find_nearest_neighbours_pace(index)
    row_pace = row_pace.mean()
    row_pace = pd.DataFrame(row_pace)
    row_pace = row_pace.T
    row_pace.index = query.index
    #pred_pace = row_pace
    return row_pace

#neighbours = find_nearest_neighbours_pace(60)
#weighted_row(neighbours)
#tester = find_nearest_neighbours_pace(60)
#weighted_rows(tester)

pace_rec(0)

,Year,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
0,2013.8,2316.638333,9.010662,NaN,NaN,7.530044,NaN,9.875405,12.203704,17.114848,9.333987,14.502265,8.865646,18.449167,9.507778,11.007317,NaN,NaN,26.44898,11.289414,12.634295,22.148519,10.801833,13.645833,13.771717,19.910341,18.24238,13.877778,19.912821,13.679705,23.054245,19.300347,13.918056,16.954023,21.930159,15.229365,22.513514,15.729461,15.263985,18.289286,10.425619,14.248526,2071.601667,7.822525,NaN,NaN,6.372763,NaN,8.975146,11.559218,16.09,8.599608,13.75055,11.918095,17.87875,8.632593,9.819837,33.917593,19.015278,23.446453,10.526827,12.388013,21.103704,8.613,10.934286,NaN,NaN,16.013333,NaN,NaN,10.971497,19.951468,19.50434,10.153241,15.431034,18.368182,NaN,19.216216,14.390909,NaN,18.476786,9.034955,13.268314


In [919]:
def weighted_rows(index):
    #print(index)
    query = npb.iloc[index]
    query = pd.DataFrame(query)
    query = query.T
    row_pace = find_nearest_neighbours_pace(index)
    row_pace = row_pace.mean()
    row_pace = pd.DataFrame(row_pace)
    row_pace = row_pace.T
    row_pace.index = query.index
    #pred_pace = row_pace
    return row_pace

#neighbours = find_nearest_neighbours_pace(60)
#weighted_row(neighbours)
#tester = find_nearest_neighbours_pace(60)
#weighted_rows(tester)

weighted_rows(0)

,Year,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
0,2013.8,2316.638333,9.010662,NaN,NaN,7.530044,NaN,9.875405,12.203704,17.114848,9.333987,14.502265,8.865646,18.449167,9.507778,11.007317,NaN,NaN,26.44898,11.289414,12.634295,22.148519,10.801833,13.645833,13.771717,19.910341,18.24238,13.877778,19.912821,13.679705,23.054245,19.300347,13.918056,16.954023,21.930159,15.229365,22.513514,15.729461,15.263985,18.289286,10.425619,14.248526,2071.601667,7.822525,NaN,NaN,6.372763,NaN,8.975146,11.559218,16.09,8.599608,13.75055,11.918095,17.87875,8.632593,9.819837,33.917593,19.015278,23.446453,10.526827,12.388013,21.103704,8.613,10.934286,NaN,NaN,16.013333,NaN,NaN,10.971497,19.951468,19.50434,10.153241,15.431034,18.368182,NaN,19.216216,14.390909,NaN,18.476786,9.034955,13.268314


In [921]:
weighted_rows(4)

,Year,time,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,AVG_Pace,time_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,AVG_Pace_pb
4,2013.7,2680.56,9.496887,NaN,NaN,8.259123,NaN,11.930227,13.629342,19.414242,11.333856,17.190097,10.828707,20.962917,11.517407,13.671789,NaN,NaN,31.068844,14.451892,15.229231,26.540741,11.7065,15.517857,21.406117,NaN,22.530605,18.159083,NaN,15.878401,30.183718,24.305556,13.064815,18.856322,23.416354,NaN,19.977477,16.547026,NaN,23.111905,12.764054,16.588845,2443.755,8.717181,NaN,NaN,7.240395,NaN,10.276634,13.009506,17.566364,9.355621,14.886019,17.66932,19.223333,9.635185,10.850894,48.917593,22.284722,23.306122,12.776306,13.229295,22.948519,10.138333,13.26619,19.578114,NaN,19.047329,17.832804,NaN,12.648677,23.37058,25.541667,12.24784,18.140805,20.711496,NaN,25.31982,16.216386,NaN,19.940476,10.656937,15.722523


####  means

In [922]:
def get_mean_pb(neighbours):
    mean = neighbours.AVG_Pace_pb.mean() 
    return mean
    
# Show sample output    
neighbours = find_nearest_neighbours(10)
get_mean_pb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


12.292952298968425

In [923]:
def get_weight_mean(neighbours):
    w_mean = neighbours.weighted_pace.sum()
    return w_mean

neighbours = find_nearest_neighbours(10)
get_weight_mean(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


12.293729641949012

In [924]:
def euc_get_weight_mean(neighbours):
    ew_mean = neighbours.weighted_pace.sum()
    return ew_mean

neighbours = euc_find_nearest_neighbours(10)
euc_get_weight_mean(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


12.312662883582897

In [926]:
def get_best_npb(neighbours):
    return neighbours.AVG_Pace.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(60)
get_best_npb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


12.826205358142095

## Get Best

In [927]:
def get_best_pb(neighbours):
    return neighbours.AVG_Pace_pb.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(5)
get_best_pb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


12.891325936857145

In [928]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_npb(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(0)
get_prediction(0, neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


11.939618101641624

# Calculate Predictions

In [929]:
def update_rows(row):
    index = int(row['best_pb_prediction'])
    row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
    row['query_pb'] = merge.iloc[index].AVG_Pace_pb
    neighbours = find_nearest_neighbours(index)
    row['mean_pb_average_pace'] = get_mean_pb(neighbours)
   # row['weighted_mean_pb_ap'] = weighted_pace(neighbours)
  #  row['best_average_pace'] = get_best_npb(neighbours)
    row['best_pb_prediction'] = get_prediction(index, neighbours)
    row['weighted_mean'] = get_weight_mean(neighbours)
    neighbours1 = euc_find_nearest_neighbours(index)
    row['euc_weighted_mean'] = euc_get_weight_mean(neighbours1)
   # row['ranked_mean'] = get_rank_mean(neighbours)
    return row

In [930]:
num_of_runs = merge.time.count()
filler_data = np.arange(0.0, num_of_runs)
df = pd.DataFrame({'best_pb_prediction':filler_data, 'mean_pb_average_pace':filler_data,  'weighted_mean':filler_data,   'euc_weighted_mean':filler_data, 'query_AVG_Pace':filler_data, 'query_pb':filler_data})

df = df.apply(lambda row: update_rows(row), axis=1)
df = df[['query_AVG_Pace', 'query_pb', 'best_pb_prediction', 'mean_pb_average_pace', 'weighted_mean', 'euc_weighted_mean']]
df

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
0,14.183180,13.589501,11.939618,13.268314,13.268791,13.182445
1,15.088612,13.589501,12.676387,14.406557,14.405696,14.389533
2,16.802140,15.121991,14.413524,15.701139,15.701881,15.564509
3,16.236556,16.326208,14.475365,15.200261,15.200349,14.962338
4,17.313847,17.965522,12.897879,15.722523,15.722237,15.716331
5,15.082644,11.879689,13.750428,14.581179,14.580432,14.225370
6,17.045035,14.525623,12.449847,15.182169,15.182254,15.201471
7,17.672900,17.528080,13.827280,15.466860,15.466478,15.404616
8,16.107976,14.311773,12.655385,15.009180,15.009177,15.189139
9,12.528339,12.765600,11.283028,11.469732,11.467746,11.457908


In [931]:
df.iloc[[66]]

,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
66,15.759533,15.384731,13.306187,14.920343,14.921178,15.343602


Comb dataframe contains the information of the gender and category of each runner. This will be used in a later notebook to test how accurate the predictions are in comparison to male, female, senior men, etc...

In [932]:
comb.join(df)

,category,gender,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
0,Senior Men (23-39),Male,14.183180,13.589501,11.939618,13.268314,13.268791,13.182445
1,Senior Men (23-39),Male,15.088612,13.589501,12.676387,14.406557,14.405696,14.389533
2,Masters Men 1 (40-49),Male,16.802140,15.121991,14.413524,15.701139,15.701881,15.564509
3,Masters Men 1 (40-49),Male,16.236556,16.326208,14.475365,15.200261,15.200349,14.962338
4,Senior Men (23-39),Male,17.313847,17.965522,12.897879,15.722523,15.722237,15.716331
5,Senior Men (23-39),Male,15.082644,11.879689,13.750428,14.581179,14.580432,14.225370
6,Senior Men (23-39),Male,17.045035,14.525623,12.449847,15.182169,15.182254,15.201471
7,Masters Men 1 (40-49),Male,17.672900,17.528080,13.827280,15.466860,15.466478,15.404616
8,Masters Men 2 (50-59),Male,16.107976,14.311773,12.655385,15.009180,15.009177,15.189139
9,Senior Women (23-39),Female,12.528339,12.765600,11.283028,11.469732,11.467746,11.457908


In [933]:
df = comb.join(df)

In [934]:
df.to_csv('../../data/weather_pace_predictions.csv')

In [935]:
predict = pd.read_csv('../../data/weather_pace_predictions.csv')

## Coverage

In [936]:
predict = len(predict)

In [937]:
predict

994

In [977]:
(predict / first_df) * 100

98.80715705765407